In [1]:
import os
import gym
import tqdm
import json
import torch
import mlflow
import pickle
import argparse
import datetime
from os import path
from dynaconf import Dynaconf
from ray.rllib.algorithms.dqn import DQN
from algorithms_with_statistics.ddqn_pber import DDQNWithMPBERAndERLogging
from algorithms_with_statistics.ddqn_per import DDQNWithMPERAndLogging
from algorithms.ddqn_pber import DDQNWithMPBER
from replay_buffer.mpber import MultiAgentPrioritizedBlockReplayBuffer
from ray.rllib.env.wrappers.atari_wrappers import wrap_deepmind
from utils import init_ray, check_path, logs_with_timeout, convert_np_arrays
from mlflow.exceptions import MlflowException
from func_timeout import FunctionTimedOut

In [ ]:
torch.manual_seed(10)

In [1]:
run_name = "examples_pber"
setting = "./settings/ddqn_per/BeamRider.yml"
log_path = "~/loggings/"
checkpoint_path = "~/checkpoints/"
sub_buffer_size= 8
single_ray = 0
with_er_logging = 1

In [3]:
if single_ray == 1:
    init_ray()
else:
    init_ray("./ray_config.yml")

2023-06-28 02:05:04,463	WARNING utils.py:636 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2023-06-28 02:05:05,673	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [5]:
settings = Dynaconf(envvar_prefix="DYNACONF", settings_files=setting)

In [8]:
mlflow.set_tracking_uri(settings.mlflow.url)
mlflow.set_experiment(experiment_name=settings.mlflow.experiment)
mlflow_client = mlflow.tracking.MlflowClient()

In [10]:
hyper_parameters = settings.dqn.hyper_parameters.to_dict()
hyper_parameters["logger_config"] = {"type": UnifiedLogger, "logdir": checkpoint_path}
print("log path: %s \n check_path: %s" % (log_path, checkpoint_path))

In [11]:
if sub_buffer_size == 0:
    # Set run object
    run_name = run_name + "_" + settings.dqn.env + "_PER_%d" % parser.parse_args().run_name
    mlflow_run = mlflow.start_run(run_name=run_name,
                                  tags={"mlflow.user": settings.mlflow.user})
    # Log parameters
    mlflow.log_params(hyper_parameters["replay_buffer_config"])
    mlflow.log_params(
        {key: hyper_parameters[key] for key in hyper_parameters.keys() if key not in ["replay_buffer_config"]})
    if with_er_logging:
        algorithm = DDQNWithMPERAndLogging(config=hyper_parameters, env=settings.dqn.env)
    else:
        algorithm = DQN(config=hyper_parameters, env=settings.dqn.env)
else:
    # Set run object
    run_name = run_name + "_" + settings.dqn.env + "_PBER_%d" % parser.parse_args().run_name
    mlflow_run = mlflow.start_run(run_name=run_name,
                                  tags={"mlflow.user": settings.mlflow.user})
    env_example = wrap_deepmind(gym.make(settings.dqn.env))
    # Log parameters
    mlflow.log_params({
        **settings.dqn.hyper_parameters.replay_buffer_config.to_dict(),
        "type": "MultiAgentPrioritizedBlockReplayBuffer",
        "sub_buffer_size": sub_buffer_size,
    })
    mlflow.log_params(
        {key: hyper_parameters[key] for key in hyper_parameters.keys() if key not in ["replay_buffer_config"]})
    # Set BER
    replay_buffer_config = {
        **settings.dqn.hyper_parameters.replay_buffer_config.to_dict(),
        "type": MultiAgentPrioritizedBlockReplayBuffer,
        "obs_space": env_example.observation_space,
        "action_space": env_example.action_space,
        "sub_buffer_size": sub_buffer_size,
        "worker_side_prioritization": False,
        "replay_sequence_length": 1,
    }
    hyper_parameters["replay_buffer_config"] = replay_buffer_config
    if with_er_logging:
        algorithm = DDQNWithMPBERAndERLogging(config=hyper_parameters, env=settings.dqn.env)
    else:
        algorithm = DDQNWithMPBER(config=hyper_parameters, env=settings.dqn.env)

In [12]:
print(algorithm.config.to_dict()["replay_buffer_config"])

In [14]:
check_path(log_path)
log_path = path.join(log_path, run_name)
check_path(log_path)
check_path(checkpoint_path)
checkpoint_path = path.join(checkpoint_path, run_name)
check_path(checkpoint_path)

In [ ]:
with open(os.path.join(checkpoint_path, "%s_config.pyl" % run_name), "wb") as f:
    _ = algorithm.config.to_dict()
    _.pop("multiagent")
    pickle.dump(_, f)
    
checkpoint_path = path.join(checkpoint_path, "results")
check_path(checkpoint_path)
mlflow.log_artifacts(checkpoint_path)

In [15]:
keys_to_extract = {"episode_reward_max", "episode_reward_min", "episode_reward_mean"}

In [ ]:
for i in tqdm.tqdm(range(1, 10000)):
    try:
        result = algorithm.train()
        time_used = result["time_total_s"]
        # statistics
        sampler = result.get("sampler_results", None)
    except:
        continue
    try:
        if i >= 10 and i % settings.log.log == 0:
            learner_data = result["info"].copy()
            if learner_data["learner"].get("time_usage", None) is not None:
                logs_with_timeout(learner_data["learner"].get("time_usage"), step=result["episodes_total"])
            learner_data.pop("learner")
            logs_with_timeout(learner_data, step=result["episodes_total"])
            _save = {key: sampler[key] for key in keys_to_extract if key in sampler}
            logs_with_timeout(_save, step=result["episodes_total"])
        if i % settings.log.log == 0:
            algorithm.save_checkpoint(checkpoint_path)
    except FunctionTimedOut:
        tqdm.tqdm.write("logging failed")
    except MlflowException:
        tqdm.tqdm.write("logging failed")
    with open(path.join(log_path, str(i) + ".json"), "w") as f:
        result["config"] = None
        json.dump(convert_np_arrays(result), f)
    if time_used >= settings.log.max_time:
        break

  0%|          | 0/999 [00:00<?, ?it/s]

In [ ]:
mlflow.log_artifacts(log_path)
mlflow.log_artifacts(checkpoint_path)
